# Welcome to the start of your adventure in Agentic AI

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Are you ready for action??</h2>
            <span style="color:#ff7800;">Have you completed all the setup steps in the <a href="../setup/">setup</a> folder?<br/>
            Have you read the <a href="../README.md">README</a>? Many common questions are answered here!<br/>
            Have you checked out the guides in the <a href="../guides/01_intro.ipynb">guides</a> folder?<br/>
            Well in that case, you're ready!!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">This code is a live resource - keep an eye out for my updates</h2>
            <span style="color:#00bfff;">I push updates regularly. As people ask questions or have problems, I add more examples and improve explanations. As a result, the code below might not be identical to the videos, as I've added more steps and better comments. Consider this like an interactive book that accompanies the lectures.<br/><br/>
            I try to send emails regularly with important updates related to the course. You can find this in the 'Announcements' section of Udemy in the left sidebar. You can also choose to receive my emails via your Notification Settings in Udemy. I'm respectful of your inbox and always try to add value with my emails!
            </span>
        </td>
    </tr>
</table>

### And please do remember to contact me if I can help

And I love to connect: https://www.linkedin.com/in/eddonner/


### New to Notebooks like this one? Head over to the guides folder!

Just to check you've already added the Python and Jupyter extensions to Cursor, if not already installed:
- Open extensions (View >> extensions)
- Search for python, and when the results show, click on the ms-python one, and Install it if not already installed
- Search for jupyter, and when the results show, click on the Microsoft one, and Install it if not already installed  
Then View >> Explorer to bring back the File Explorer.

And then:
1. Click where it says "Select Kernel" near the top right, and select the option called `.venv (Python 3.12.9)` or similar, which should be the first choice or the most prominent choice. You may need to choose "Python Environments" first.
2. Click in each "cell" below, starting with the cell immediately below this text, and press Shift+Enter to run
3. Enjoy!

After you click "Select Kernel", if there is no option like `.venv (Python 3.12.9)` then please do the following:  
1. On Mac: From the Cursor menu, choose Settings >> VS Code Settings (NOTE: be sure to select `VSCode Settings` not `Cursor Settings`);  
On Windows PC: From the File menu, choose Preferences >> VS Code Settings(NOTE: be sure to select `VSCode Settings` not `Cursor Settings`)  
2. In the Settings search bar, type "venv"  
3. In the field "Path to folder with a list of Virtual Environments" put the path to the project root, like C:\Users\username\projects\agents (on a Windows PC) or /Users/username/projects/agents (on Mac or Linux).  
And then try again.

Having problems with missing Python versions in that list? Have you ever used Anaconda before? It might be interferring. Quit Cursor, bring up a new command line, and make sure that your Anaconda environment is deactivated:    
`conda deactivate`  
And if you still have any problems with conda and python versions, it's possible that you will need to run this too:  
`conda config --set auto_activate_base false`  
and then from within the Agents directory, you should be able to run `uv python list` and see the Python 3.12 version.

In [16]:
# First let's do an import. If you get an Import Error, double check that your Kernel is correct..

from dotenv import load_dotenv


In [17]:
# Next it's time to load the API keys into environment variables
# If this returns false, see the next cell!

load_dotenv(override=True)

True

### Wait, did that just output `False`??

If so, the most common reason is that you didn't save your `.env` file after adding the key! Be sure to have saved.

Also, make sure the `.env` file is named precisely `.env` and is in the project root directory (`agents`)

By the way, your `.env` file should have a stop symbol next to it in Cursor on the left, and that's actually a good thing: that's Cursor saying to you, "hey, I realize this is a file filled with secret information, and I'm not going to send it to an external AI to suggest changes, because your keys should not be shown to anyone else."

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Final reminders</h2>
            <span style="color:#ff7800;">1. If you're not confident about Environment Variables or Web Endpoints / APIs, please read Topics 3 and 5 in this <a href="../guides/04_technical_foundations.ipynb">technical foundations guide</a>.<br/>
            2. If you want to use AIs other than OpenAI, like Gemini, DeepSeek or Ollama (free), please see the first section in this <a href="../guides/09_ai_apis_and_ollama.ipynb">AI APIs guide</a>.<br/>
            3. If you ever get a Name Error in Python, you can always fix it immediately; see the last section of this <a href="../guides/06_python_foundations.ipynb">Python Foundations guide</a> and follow both tutorials and exercises.<br/>
            </span>
        </td>
    </tr>
</table>

In [18]:
# Check the key - if you're not using OpenAI, check whichever key you're using! Ollama doesn't need a key.

import os
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set - please head to the troubleshooting guide in the setup folder")
    


OpenAI API Key exists and begins sk-proj-


In [19]:
# And now - the all important import statement
# If you get an import error - head over to troubleshooting in the Setup folder
# Even for other LLM providers like Gemini, you still use this OpenAI import - see Guide 9 for why

from openai import OpenAI

In [20]:
# And now we'll create an instance of the OpenAI class
# If you're not sure what it means to create an instance of a class - head over to the guides folder (guide 6)!
# If you get a NameError - head over to the guides folder (guide 6)to learn about NameErrors - always instantly fixable
# If you're not using OpenAI, you just need to slightly modify this - precise instructions are in the AI APIs guide (guide 9)

openai = OpenAI()

In [21]:
# Create a list of messages in the familiar OpenAI format

messages = [{"role": "user", "content": "What is 2+2?"}]

In [22]:
# And now call it! Any problems, head to the troubleshooting guide
# This uses GPT 4.1 nano, the incredibly cheap model
# The APIs guide (guide 9) has exact instructions for using even cheaper or free alternatives to OpenAI
# If you get a NameError, head to the guides folder (guide 6) to learn about NameErrors - always instantly fixable

response = openai.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages
)

print(response.choices[0].message.content)


2 + 2 equals 4.


In [23]:
# And now - let's ask for a question:

question = "Please propose a hard, challenging question to assess someone's IQ. Respond only with the question."
messages = [{"role": "user", "content": question}]


In [24]:
# ask it - this uses GPT 4.1 mini, still cheap but more powerful than nano

response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=messages
)

question = response.choices[0].message.content

print(question)


If 5 machines take 5 minutes to make 5 widgets, how long would 100 machines take to make 100 widgets?


In [25]:
# form a new messages list
messages = [{"role": "user", "content": question}]


In [26]:
# Ask it again

response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=messages
)

answer = response.choices[0].message.content
print(answer)


Let's analyze the problem step-by-step:

- 5 machines take 5 minutes to make 5 widgets.
  
From this, we can find how long it takes one machine to make one widget.

---

**Step 1: Find the rate of one machine**

Since 5 machines make 5 widgets in 5 minutes, that means:

- Total widgets made per minute by 5 machines = \( \frac{5 \text{ widgets}}{5 \text{ minutes}} = 1 \text{ widget/minute} \)

- So, 5 machines make 1 widget per minute.

- Therefore, 1 machine makes \( \frac{1}{5} \) widgets per minute.

---

**Step 2: Calculate time for 100 machines to make 100 widgets**

- 1 machine’s rate = \( \frac{1}{5} \) widgets per minute.

- 100 machines’ rate = \( 100 \times \frac{1}{5} = 20 \) widgets per minute.

- To make 100 widgets at 20 widgets per minute, the time needed is:

\[
\frac{100 \text{ widgets}}{20 \text{ widgets/minute}} = 5 \text{ minutes}
\]

---

**Answer:**

\[
\boxed{5 \text{ minutes}}
\]

So, 100 machines will take 5 minutes to make 100 widgets.


In [27]:
from IPython.display import Markdown, display

display(Markdown(answer))



Let's analyze the problem step-by-step:

- 5 machines take 5 minutes to make 5 widgets.
  
From this, we can find how long it takes one machine to make one widget.

---

**Step 1: Find the rate of one machine**

Since 5 machines make 5 widgets in 5 minutes, that means:

- Total widgets made per minute by 5 machines = \( \frac{5 \text{ widgets}}{5 \text{ minutes}} = 1 \text{ widget/minute} \)

- So, 5 machines make 1 widget per minute.

- Therefore, 1 machine makes \( \frac{1}{5} \) widgets per minute.

---

**Step 2: Calculate time for 100 machines to make 100 widgets**

- 1 machine’s rate = \( \frac{1}{5} \) widgets per minute.

- 100 machines’ rate = \( 100 \times \frac{1}{5} = 20 \) widgets per minute.

- To make 100 widgets at 20 widgets per minute, the time needed is:

\[
\frac{100 \text{ widgets}}{20 \text{ widgets/minute}} = 5 \text{ minutes}
\]

---

**Answer:**

\[
\boxed{5 \text{ minutes}}
\]

So, 100 machines will take 5 minutes to make 100 widgets.

# Congratulations!

That was a small, simple step in the direction of Agentic AI, with your new environment!

Next time things get more interesting...

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Now try this commercial application:<br/>
            First ask the LLM to pick a business area that might be worth exploring for an Agentic AI opportunity.<br/>
            Then ask the LLM to present a pain-point in that industry - something challenging that might be ripe for an Agentic solution.<br/>
            Finally have 3 third LLM call propose the Agentic AI solution. <br/>
            We will cover this at up-coming labs, so don't worry if you're unsure.. just give it a try!
            </span>
        </td>
    </tr>
</table>

In [29]:
# First create the messages:

messages = [{"role": "user", "content": "Could you pick a business area that might be worth exploring for an Agentic AI opportunity?"}]

# Then make the first call:

response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=messages
)

# Then read the business idea:

business_idea = response.choices[0].message.content
print(business_idea)

# And repeat! In the next message, include the business idea within the message

Certainly! One promising business area for an Agentic AI opportunity is **Personalized Healthcare Management**.

### Why Personalized Healthcare Management?

1. **Complexity and Personalization Needs**  
   Healthcare involves vast amounts of data—from patient histories and genetics to lifestyle and environment—that require nuanced analysis. An agentic AI can autonomously gather, interpret, and act on this data to deliver tailored health recommendations, treatment plans, and proactive monitoring.

2. **Continuous and Dynamic Decision-Making**  
   Patients’ health conditions evolve over time, demanding continuous adjustment of care strategies. An agentic AI could autonomously manage medication schedules, suggest lifestyle changes, or detect early signs of complications, adapting its guidance dynamically.

3. **Integration with Wearable and IoT Devices**  
   With an increasing number of health monitoring devices, agentic AI can aggregate real-time sensor data, provide alerts, and inter

In [30]:
messages = [{"role": "user", "content": "Could you present a pain-point in Personalized Healthcare Management - something challenging that might be ripe for an Agentic solution?"}]

# Then make the first call:

response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=messages
)

# Then read the business idea:

challenging_idea = response.choices[0].message.content
print(challenging_idea)

Certainly! One significant pain point in Personalized Healthcare Management is **the integration and interpretation of heterogeneous health data from multiple sources to provide timely, individualized care recommendations**.

### Challenge:
Patients generate vast amounts of health-related data from diverse channels—clinical records, wearable devices, genomic information, patient-reported outcomes, and even social determinants of health. However, this data is often siloed, inconsistently formatted, and overwhelming for both patients and healthcare providers to interpret. Consequently:

- Clinicians may struggle to obtain a holistic, real-time view of a patient’s health status.
- Personalized treatment adjustments can be delayed or suboptimal.
- Patients receive generalized rather than truly tailored healthcare advice, reducing engagement and adherence.

### Why This is Ripe for an Agentic Solution:
An **agentic AI system**—one that can autonomously gather, synthesize, and reason over mu